In [1]:
import torch
from torch import nn
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from LSTM import *
import pandas as pd
import numpy as np
import torch.utils.data as utils
import time
import os
import pickle

import pdb

# For the notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

torch.manual_seed(1)

In [3]:
gt_test_coord_1 = pickle.load( open( "../import_dataset_2/test/gt_test_coord_1.pkl", "rb" ) )
in_test_coord_1 = pickle.load( open( "../import_dataset_2/test/in_test_coord_1.pkl", "rb" ) )
inputs_test_1 = pickle.load( open( "../import_dataset_2/test/inputs_test_1.pkl", "rb" ) )

gt_test_coord_2 = pickle.load( open( "../import_dataset_2/test/gt_test_coord_2.pkl", "rb" ) )
in_test_coord_2 = pickle.load( open( "../import_dataset_2/test/in_test_coord_2.pkl", "rb" ) )
inputs_test_2 = pickle.load( open( "../import_dataset_2/test/inputs_test_2.pkl", "rb" ) )

gt_test_coord_3 = pickle.load( open( "../import_dataset_2/test/gt_test_coord_3.pkl", "rb" ) )
in_test_coord_3 = pickle.load( open( "../import_dataset_2/test/in_test_coord_3.pkl", "rb" ) )
inputs_test_3 = pickle.load( open( "../import_dataset_2/test/inputs_test_3.pkl", "rb" ) )

coord_ind = np.arange(0,2)
inputs_test_1 = inputs_test_1[:,:,coord_ind]
inputs_test_2 = inputs_test_2[:,:,coord_ind]
inputs_test_3 = inputs_test_3[:,:,coord_ind]

inputs_test = np.concatenate((inputs_test_1,inputs_test_2,inputs_test_3),axis=1)
in_test_coord = np.concatenate((in_test_coord_1,in_test_coord_2,in_test_coord_3),axis=1)
gt_test_coord = np.concatenate((gt_test_coord_1,gt_test_coord_2,gt_test_coord_3),axis=1)
inputs_test.shape

(10, 1718, 2)

##### Linear

In [4]:
def linearpred(inputs):
    output_coord = np.zeros((inputs.shape[0]+1,
                             inputs.shape[1],inputs.shape[2]))
    output_coord[0,:,:]=inputs[9,:,:]
    for i in range(inputs.shape[1]):
        if (inputs[9,i,0]-inputs[8,i,0])==0:
            output_coord[1:,i,0] =inputs[9,i,0]*np.ones(10)
            diff = inputs[9,i,1]-inputs[8,i,1]
            output_coord[1:,i,0] = range(10)*diff+inputs[9,i,1]
        else:
            slope = (inputs[9,i,1]-inputs[8,i,1])/(inputs[9,i,0]-inputs[8,i,0])
            ordor = inputs[9,i,1] - slope*inputs[9,i,0]
            diff = inputs[9,i,0] - inputs[8,i,0]
            output_coord[1:,i,0] = range(10)*diff+inputs[9,i,0]
            output_coord[1:,i,1] = slope*output_coord[1:,i,0]+ordor
    return output_coord

In [5]:
output_coord_1 = linearpred(inputs_test_1)
dist = np.zeros(inputs_test_1.shape[1])
for i in range(inputs_test_1.shape[1]):
    dist[i] = np.sqrt(sum((gt_test_coord_1[10,i,:]-output_coord_1[10,i,:])**2))
    
final_coord_error = np.mean(dist)
print('The final distance between the ground trought and the predicted coordinates is :',
      final_coord_error.round(3))

avr = np.zeros((inputs_test_1.shape[1],11))
for i in range(inputs_test_1.shape[1]):
    for j in range(11):
        avr[i,j] = np.sqrt(sum((gt_test_coord_1[j,i,:]-output_coord_1[j,i,:])**2))

average = np.mean(np.mean(avr,1))
print('The average error between the ground trought and the predicted coordinates is :',
      average.round(3))        

The final distance between the ground trought and the predicted coordinates is : 0.322
The average error between the ground trought and the predicted coordinates is : 0.141


In [6]:
output_coord_2 = linearpred(inputs_test_2)

dist = np.zeros(inputs_test_2.shape[1])
for i in range(inputs_test_2.shape[1]):
    dist[i] = np.sqrt(sum((gt_test_coord_2[10,i,:]-output_coord_2[10,i,:])**2))

final_coord_error = np.mean(dist)
print('The final distance between the ground truth and the predicted coordinates is :',final_coord_error.round(3))

avr = np.zeros((inputs_test_2.shape[1],11))
for i in range(inputs_test_2.shape[1]):
    for j in range(11):
        avr[i,j] = np.sqrt(sum((gt_test_coord_2[j,i,:]-output_coord_2[j,i,:])**2))

average = np.mean(np.mean(avr,1))
print('The average error between the ground truth and the predicted coordinates is :',average.round(3))

The final distance between the ground truth and the predicted coordinates is : 0.93
The average error between the ground truth and the predicted coordinates is : 0.541


In [7]:
output_coord_3 = linearpred(inputs_test_3)      

dist = np.zeros(inputs_test_3.shape[1])
for i in range(inputs_test_3.shape[1]):
    dist[i] = np.sqrt(sum((gt_test_coord_3[10,i,:]-output_coord_3[10,i,:])**2))

final_coord_error = np.mean(dist)
print('The final distance between the ground truth and the predicted coordinates is :',
      final_coord_error.round(3))


avr = np.zeros((inputs_test_3.shape[1],11))
for i in range(inputs_test_3.shape[1]):
    for j in range(11):
        avr[i,j] = np.sqrt(sum((gt_test_coord_3[j,i,:]-output_coord_3[j,i,:])**2))

average = np.mean(np.mean(avr,1))
print('The average error between the ground trought and the predicted coordinates is :',
      average.round(3))

The final distance between the ground truth and the predicted coordinates is : 1.457
The average error between the ground trought and the predicted coordinates is : 0.651


In [8]:
output_coord = linearpred(inputs_test)

dist = np.zeros(inputs_test.shape[1])
for i in range(inputs_test.shape[1]):
    dist[i] = np.sqrt(sum((gt_test_coord[10,i,:]-output_coord[10,i,:])**2))

final_coord_error = np.mean(dist)
print('The final distance between the ground truth and the predicted coordinates is :',
      final_coord_error.round(3))


avr = np.zeros((inputs_test.shape[1],11))
for i in range(inputs_test.shape[1]):
    for j in range(11):
        avr[i,j] = np.sqrt(sum((gt_test_coord[j,i,:]-output_coord[j,i,:])**2))

average = np.mean(np.mean(avr,1))
print('The average error between the ground trought and the predicted coordinates is :',
      average.round(3))

The final distance between the ground truth and the predicted coordinates is : 0.982
The average error between the ground trought and the predicted coordinates is : 0.512


In [ ]:
output_coord = np.zeros((inputs_test.shape[0]+1,inputs_test.shape[1],inputs_test.shape[2]))

In [ ]:
output_coord.shape,inputs_test.shape

In [ ]:
inputs_test_1.shape[1]